In [1]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests
import json
import gdown

# pd.set_option('display.max_rows', None)
# pd.set_option('mode.chained_assignment',None)

In [2]:
gdown.download(id = '1h0HiImmEonZCBUcRXOwYAi9g39nYs-RX')

Downloading...
From: https://drive.google.com/uc?id=1h0HiImmEonZCBUcRXOwYAi9g39nYs-RX
To: /content/georef-australia-state-suburb.csv
100%|██████████| 53.5M/53.5M [00:00<00:00, 77.4MB/s]


'georef-australia-state-suburb.csv'

In [3]:
df = pd.read_csv('/content/georef-australia-state-suburb.csv',sep=';')
del df['Geo Shape'], df['Geo Point'], df['Type'], df['Year'], df['Iso 3166-3 Area Code']

* The website had only sugestions to pull data from.
* so we have a sugestion genrator with 3 charecters.
* its total combinations lengeth were 17576

In [14]:
from itertools import product
from string import ascii_lowercase
keywords = tuple(map(''.join, product(ascii_lowercase, repeat=3)))
print(len(keywords))

17576


In [15]:
area_list = df['Official Name Local Government Area']

In [16]:
headers = {
    'authority': 'www.domain.com.au',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9',
    'content-type': 'application/json',
    # 'cookie': 'domain-mixpanel-id_ab0bde70050c3eabaaf8824402fa01e0=$device:188995b514457e-03d062656d4fe5-26031d51-e0985-188995b5145100b; DEVICE_SESSIONID=a023599d-f7d9-46eb-b321-f121e875624e; searchSOI=nsw; bm_sz=E5C3168193AD41FE541960D95436255D~YAAQX+hUuDpcuJaIAQAA3jpJrhTax8oHVn/x4xpTKyn+pZ3tBypaiFN+fhR/FkbUiUtsbtuptrKHlrQVtLONesMfl+u22kTH6Oe8fVzvoXh0i9/cktskT2ksu8Pl/sK2ke2twQKuLm7o72/j1Zlmj/NztVuvVJHbLx2PWFbJJSVaJroFVp94sNdqGqmQI4m79RXMWA2cJHdE/EMHzxxRm3eeZ0wK365YXrDIM1MggRFMgn8svljDLLWJWCpLyPWBw7cbNNfG4viqhJZSnPCtngE5yd2wCks5OL3m4UX0p2YiWKhVvjefSvZJSq/wRTYWAiQwvgnyyp5HNofRq20=~3421252~4473143; bm_mi=D952F10DCA95880574A8BC722F2EB220~YAAQX+hUuCPXuJaIAQAA0bBarhSxMOjumckKnKAlJGqyac9HJv7nNQQY018qrEOD6li5mlKJJdlDNNnQMNimYXRtxDPmKz5LEZvmj16ofM4zzp1LEoVU9F6EYvK6d4Ip0JJi5ZveAW787x1wVwMpepTQdUBDRpiPzNBKDnV0+8dI12TDQIIgIetF7fMeiQoqfEon4lKfpkhnH1TxQtk4qj2a/LWB2L3d4lue7GxH6gjdtlP2fM1aZcux9RSVPu1DVT/kxVCwUYnQ/rqfYsszfYcWGD+MbOI6nSKiHeuMXKb70P8IDPt9MNraiAvYW9bte2h28jNHksjvzb1xXKvr~1; ak_bmsc=1DE2C239D9A96F1142226B5FA5500183~000000000000000000000000000000~YAAQX+hUuBnYuJaIAQAAcNxarhTQh3sbnvdeNNlrfoGWlFkM3eHfk14mHOm1PZyIU0gmYFxPO9/qdHza1NrXcHuNpWrNwVvbg1dISowLTtF7HsRvUEX7jU+wau+gQjX6thKlHYUF3K50ITyCdJE8dscqRM5NoYZj7uO0M/LIZpD6beritGyU896dEKD/2dPy/o7CGykFe56byEq8XJv3tFoJyQb1fIW8twUkzznRLOKajOXSd/klW/N6rLuvXGJjoFFfWEvBgi4VI4eMtUuGMiQZRQohNzTQ8RbVyI6r+VvPpSIlTqirB6rVBlk2MpvrWdIG6luUwoGnR2ddw6TIAvxcRK2tt3aNdHT84ggGEOn6A+UM+KndYFpWCTHs20VPTUkN2e44Zx5xqTS3zSGMbqvfWarNWtaOtm40CUxLtNdfXgqWJCEytUPJPM0/; bm_sv=A248E62E876DC5FACA1C1B94D0727478~YAAQX+hUuIa9uZaIAQAARvZ7rhQmRelJL4ghZ5opdMFxpF0zmhQLqwSZ2hKhHDu+nS4kBVQLt+JtTF4BO2IMCRWdw76Zsbhp6siOtsux3Rx/s71QUW1TztDbZE/4UyxlV02YXRDKxI2QwV8vQOa9FdGp43thdH3hsqngJNLgI4qBF7/whqCTKRIM23c7wzHh6FMDDWanOVq+Olkac3fGqtE0bH3uK9OxwL5iw7WT6qIbRb9vtC1odx86FasDcBAT5Inf8w==~1; _abck=1DD95BDDF3FCA5D055B5FB003FEBDFF4~0~YAAQX+hUuGW+uZaIAQAAPv97rgokC/jffin+CGwE6ip6uInGb+3nw3Eibix6Nd7g4m/9XzeDvpAJRj3jl30Hp2K6KBS9nLDQrwOx0+O1uxtdrvEu0LRI+e/paHjKnl/Rtmqh2JpWCqTtljFKgjFFr7mVhgDxcSwvuIJZ6YsSOZ05ofNiw9yfkHF0JcAIYYUSNTqgD/GG/RKkcW7xrE1LAxJutk06NJNXOo0WfGpnI+c3Kx6P5YU7nBvve+hi7aeCAwBJY5TSOypVZtDAdCrbUmbzYQjeJcL/DAr6VEGFeL8DOee3iIhi/X7wGnk/gl9cJsOrEQfx54sYX5JlPBMef7GBkKsS/xTbR5HAXHi+hj/OdKa+cwfslumdx7yqTtYKdtqb6jtWQWqQBtQE~-1~-1~1686552014',
    'origin': 'https://www.domain.com.au',
    'referer': 'https://www.domain.com.au/suburb-profile/',
    'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Brave";v="114"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
}

In [17]:
def extract_first_name(name):
    # Remove special characters using regex
    cleaned_name = re.sub('[^a-zA-Z ]', '', name)
    # Extract the first name
    first_name = cleaned_name.split()[0]
    return first_name

def extract_first_names(names):
    # Split the names by comma
    name_list = names.split(',')
    # Extract the first name from each name
    first_names = [extract_first_name(name) for name in name_list]
    return first_names

In [20]:
unique_names = set()
for sub_urb in keywords:
  sub_urb = extract_first_names(sub_urb)
  try:
    json_data = {
        'variables': {
            'input': sub_urb[0],
            'maxResults': 2000,
            'locationTypes': [
                'SUBURB',
            ],
        },
        'query': '\n        query SuggestLocations(\n          $input: String!\n          $locationTypes: [LocationSuggestionType!]\n          $maxResults: Int\n        ) {\n          suggestLocations(\n            prefixText: $input,\n            locationTypes: $locationTypes,\n            maxResults: $maxResults,\n            exactMatchesToTop: true) {\n            __typename\n            ... on SuburbSuggestion {\n              label: displayName\n              value: nameSlug\n            }\n            ... on AreaSuggestion {\n              label: displayName\n              value: nameSlug\n            }\n            ... on RegionSuggestion {\n              label: displayName\n              value: nameSlug\n            }\n            ... on SchoolSuggestion {\n              label: displayName\n              value: nameSlug\n            }\n          }\n         }\n       ',
    }

    response = requests.post('https://www.domain.com.au/graphql', headers=headers, json=json_data)

    if response.status_code == 200:
        data = response.json()
        length_of_data= len(data['data']['suggestLocations'])
        for i in range(length_of_data):
          value = data['data']['suggestLocations'][i]['value']
          unique_names.add(value)
  except:   
    None     


* trying to collect the sugestions we have collected 15879 suburbs instard of 15303

In [23]:
print(len(unique_names))

15879


In [22]:
df = pd.DataFrame(unique_names)
df.to_csv('urls.csv') 

# merging the collected data.

In [2]:
gdown.download(id = '13dZkmjLyJLe4qETi_Ue9T1GF_n1yAhOD')
gdown.download(id = '18-0ag4Xgje-NR7UoesAO_5A-K4EHnr3K')
gdown.download(id = '1wkVMZ6WNPM_eO448CSbIS0rfFf2wODv9')
gdown.download(id = '14qMXskFjQmYRpxOT3MkI0TV4PbaCTg87')
gdown.download(id = '1VZAZ41a4QhFWkAL1uW84_FCllQ8PjrPd')

Downloading...
From: https://drive.google.com/uc?id=13dZkmjLyJLe4qETi_Ue9T1GF_n1yAhOD
To: /content/df_urbdat1.csv
100%|██████████| 114k/114k [00:00<00:00, 3.64MB/s]
Downloading...
From: https://drive.google.com/uc?id=18-0ag4Xgje-NR7UoesAO_5A-K4EHnr3K
To: /content/df_urbdat2.csv
100%|██████████| 111k/111k [00:00<00:00, 2.26MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wkVMZ6WNPM_eO448CSbIS0rfFf2wODv9
To: /content/df_urbdat3.csv
100%|██████████| 119k/119k [00:00<00:00, 2.15MB/s]
Downloading...
From: https://drive.google.com/uc?id=14qMXskFjQmYRpxOT3MkI0TV4PbaCTg87
To: /content/df_urbdat5.csv
100%|██████████| 116k/116k [00:00<00:00, 2.36MB/s]
Downloading...
From: https://drive.google.com/uc?id=1VZAZ41a4QhFWkAL1uW84_FCllQ8PjrPd
To: /content/df_urbdat4.csv
100%|██████████| 122k/122k [00:00<00:00, 2.36MB/s]


'df_urbdat4.csv'

In [3]:
df1 = pd.read_csv('/content/df_urbdat1.csv')
df2 = pd.read_csv('/content/df_urbdat2.csv')
df3 = pd.read_csv('/content/df_urbdat3.csv')
df4 = pd.read_csv('/content/df_urbdat4.csv')
df5 = pd.read_csv('/content/df_urbdat5.csv')

In [4]:
suburbdf = pd.DataFrame()
suburbdf = pd.concat([suburbdf,df1],ignore_index =True)
suburbdf = pd.concat([suburbdf,df2],ignore_index =True)
suburbdf = pd.concat([suburbdf,df3],ignore_index =True)
suburbdf = pd.concat([suburbdf,df4],ignore_index =True)
suburbdf = pd.concat([suburbdf,df5],ignore_index =True)
suburbdf 

,Bedrooms,Type,Median Price,Avg Days on Market,Clearance Rate,Sold This Year,POPULATION,AVERAGE AGE,OWNER,RENTER,FAMILY,SINGLE,SUBURB
0,3,House,-,-,-,1,77,40 to 59,71%,29%,56%,44%,daradgee-qld-4860
1,2,House,-,-,-,5,"4,215",20 to 39,57%,43%,44%,56%,gordon-park-qld-4031
2,3,House,$1.208m,-,64%,16,"4,215",20 to 39,57%,43%,44%,56%,gordon-park-qld-4031
3,4,House,$1.513m,-,62%,17,"4,215",20 to 39,57%,43%,44%,56%,gordon-park-qld-4031
4,5,House,$1.85m,-,62%,11,"4,215",20 to 39,57%,43%,44%,56%,gordon-park-qld-4031
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8325,5,House,-,-,-,1,374,40 to 59,74%,26%,55%,45%,veresdale-qld-4285
8326,2,House,-,-,-,2,"8,226",40 to 59,85%,15%,57%,43%,horsley-nsw-2530
8327,3,House,$795k,25 days,-,31,"8,226",40 to 59,85%,15%,57%,43%,horsley-nsw-2530
8328,4,House,$990k,44 days,-,45,"8,226",40 to 59,85%,15%,57%,43%,horsley-nsw-2530


In [5]:
suburbdf['Date'] = '13/6/2023'

Remember to upload Data_extracted in gdrive.
and also count the missing data.

In [ ]:
#suburbdf.to_csv('data_extracted.csv', index=False)

In [6]:
len(suburbdf['SUBURB'].unique())

2066

In [7]:
gdown.download(id = '1dQonVWbqhlyBFC5P7z2P2EOt_lHoBilB')
data_extracted = pd.read_csv('/content/data_extracted.csv')
data_extracted = pd.concat([data_extracted,suburbdf],ignore_index =True)

Downloading...
From: https://drive.google.com/uc?id=1dQonVWbqhlyBFC5P7z2P2EOt_lHoBilB
To: /content/data_extracted.csv
100%|██████████| 1.34M/1.34M [00:00<00:00, 10.5MB/s]


In [8]:
data_extracted.to_csv('data_extracted.csv', index=False)